In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
np.random.seed(42)

In [2]:
with open('../assets/Model_2_pipe.pkl', 'rb') as f:
    Model_2_pipe = pickle.load(f)
    
with open('../assets/Model_4_pipe.pkl', 'rb') as f:
    Model_4_pipe = pickle.load(f)
    
with open('../assets/Model_5_pipe.pkl', 'rb') as f:
    Model_5_pipe = pickle.load(f)
    
with open('../assets/columns.pkl', 'rb') as f:
    columns = pickle.load(f)

In [3]:
kaggle = pd.read_csv('../data/test.csv', index_col='Id')

In [4]:
cols = kaggle.columns
cols = cols.map(lambda x: x.replace(' ', '_') if isinstance(x, str) else x)
kaggle.columns = cols

kaggle.drop('PID', 1, inplace=True)

In [5]:
def replace_na_none(columns):
    for x in columns:
        kaggle[x].fillna("None", inplace=True)

In [6]:
replace_na_none(["Alley","Mas_Vnr_Type","Bsmt_Cond","Bsmt_Qual","Bsmt_Exposure","BsmtFin_Type_1","BsmtFin_Type_2","Fireplace_Qu","Garage_Type","Garage_Yr_Blt","Garage_Finish","Garage_Qual","Garage_Cond","Pool_QC","Fence","Misc_Feature"])

In [7]:
def data_cleaning():
    kaggle.dropna(subset=['Bsmt_Full_Bath'],inplace=True)
    kaggle['Garage_Cars'].fillna(0.0,inplace=True)
    kaggle['Garage_Area'].fillna(0.0,inplace=True)
    kaggle['Mas_Vnr_Area'].fillna(0.0,inplace=True)
    kaggle['Lot_Frontage'].fillna(0.0,inplace=True)
    kaggle.drop(['BsmtFin_SF_1', 'BsmtFin_SF_2', 'Bsmt_Unf_SF'],axis=1,inplace=True)
    kaggle.drop(['1st_Flr_SF', '2nd_Flr_SF', 'Low_Qual_Fin_SF'],axis=1,inplace=True)
    kaggle.drop(['Bedroom_AbvGr'],axis=1,inplace=True)

In [8]:
data_cleaning()

In [9]:
kaggle = pd.get_dummies(kaggle, columns=list(kaggle.select_dtypes(exclude=['int64','float64']).columns))

In [10]:
missing = [x for x in columns if x not in kaggle.columns]
for x in missing:
    kaggle.insert(1,x,0)

In [11]:
kaggle = kaggle[columns]

In [12]:
def submit(preds,name):
    submission = pd.DataFrame(np.expm1(preds), index=kaggle.index, columns=['SalePrice'])
    submission.sort_index(inplace=True)
    submission.to_csv(name)

In [13]:
submit(Model_2_pipe.predict(kaggle),'../data/kaggle_1.csv')
submit(Model_4_pipe.predict(kaggle),'../data/kaggle_2.csv')
submit(Model_5_pipe.predict(kaggle),'../data/kaggle_3.csv')